In [1]:
'''Import all necessary library files'''
import pandas as pd
import nltk
import csv
import re
import sys
import math
import operator
import collections
from math import log
from nltk.corpus import stopwords
import pickle
from __future__ import division
import matplotlib.pyplot as plt

In [2]:
cd Senti-Analysis

C:\Users\Praveen\Desktop\CS-579-Project\Senti-Analysis


**Data split into positive and negative:**

We split our dataset by considering the reviews with scores above 65 to be Positive and those ones that are below 65 to be considered as negative. Thus we create a new column called as 'score' and then we populate it by using binary values according to the nature of its Sentiment score given by the critic. Thus a review with Sentiment score greater than '65' would have a score of '1' and all others would have '0'.

In [3]:
'''Place the reviews.txt file in the working directory'''
df = pd.read_csv("reviews.txt",sep='\t')
df['score'] = [1 if x > 65 else 0 for x in df['Sentiment']]
df.head()

,﻿Review,Sentiment,score
0,That such intelligence could be contained in a...,100,1
1,Shines with a kind of inspired madness.,100,1
2,Here is the most passionate and tender love st...,100,1
3,"One of the greatest of all American films, but...",100,1
4,A remarkable film.,100,1


In [4]:
'''Create two separate dataframes based on Sentiment scores'''
pos = df[df['Sentiment'] > 65]
neg = df[df['Sentiment'] < 65]

In [5]:
pos = pos.drop('Sentiment', 1)
neg = neg.drop('Sentiment',1)
print len(pos)
print len(neg)

21027
22843


In [6]:
'''Dividing the entire dataset into new training and testing dataframes'''
pos_train = pos.head(10000)
neg_train = neg.head(10000)
pos_test = pos.head(len(pos)-len(pos_train))
neg_test = neg.head(len(neg)-len(neg_train))
print len(pos_test)
print len(neg_test)

11027
12843


In [7]:
pos_train.head()

,﻿Review,score
0,That such intelligence could be contained in a...,1
1,Shines with a kind of inspired madness.,1
2,Here is the most passionate and tender love st...,1
3,"One of the greatest of all American films, but...",1
4,A remarkable film.,1


In [8]:
neg_train.head()

,﻿Review,score
1412,Exhibiting high spirits and a crazed comic ene...,0
1413,There is something repulsive and manipulative ...,0
1414,The part that needs work didn't cost money. It...,0
1415,The movie does have charm and moments of humor...,0
1416,"Sandler, at the center, is a distraction; he s...",0


In [9]:
'''Performing Concatenation of both positive and negative training dataframes'''
frames = [pos_train,neg_train]
result = pd.concat(frames)

In [10]:
'''Printing the length of training dataframe and then re-ordering the dataframe columns'''
print len(result)
result = result.sort_index(axis=1)

20000


In [11]:
#Write the data to a temporary csv file
result.to_csv("test-ex.csv", sep=',')

In [12]:
'''Total positive and negative test dataframes'''
print len(pos_test)
print len(neg_test)

11027
12843


In [13]:
'''Concatenation of both positive and negative test dataframes together into a single dataframe 
    and writing it to a csv file.'''

frames2 = [pos_test,neg_test]
result2 = pd.concat(frames2)
result2 = result2.drop('score',1)
result2.to_csv("test-ex2.csv",sep=',')

In [14]:
'''Removing unwanted columns and storing the data into a new csv file'''

with open("test-ex.csv","rb") as source:
    rdr= csv.reader( source )
    with open("train.csv","wb") as result:
        wtr= csv.writer( result )
        for r in rdr:
            wtr.writerow( (r[1], r[2]) )

In [15]:
'''Modules to perform text pre-processing'''
# Remove Punctuations
def removePunctuation(input_list):
    punctuation=re.compile(r'[,./?!":;|-]')
    punct_remove = [punctuation.sub(" ", word) for word in input_list]
    return punct_remove

# Remove Stop Words
def removeStopWords(input_list):
    stop=stopwords.words('english')
    #We try to preserve not,nor and no as these may prove to be influential during classification.
    stop.remove('not')
    stop.remove('nor')
    stop.remove('no')
    stop.append('')
    
    for s in stop:
        while s in input_list:
            input_list.remove(s)
    return input_list


**Training our classification model:**

We read each line of our training data stored under the name of train.csv file and we carry out the initial tokenization functionalities. We then perform feature extraction procedures to fill out the positive and negative feature dictionaries, by reading each review with reference to the actual true label score of that particular review. To be precise, when we read a review whose true label score is 1 we then try to insert those tokenized words into the positive feature dictionary along with their corresponding frequency of word occurrence. Likewise we tend to populate our negative feature dictionary if the actual true label of a given review is 0.

In [16]:
def classify_train(filename):
    
    pos_dic={}
    neg_dic={}
    tot_dic={}
    pos_review_count=0
    neg_review_count=0
    total=0
    flag=False
    

    #Reading the Reviews from Training Data and Building Feature Dictionary

    with open (filename,'rb') as training_file:
        training_data=csv.reader(training_file)

        for row in training_data:
            #Using this flag to skip the first row in the csv
            if flag==False:
                flag=True
            else:
                total+=1
                word_list=re.split('\s+',row[1].lower())           
                word_list_punct=removePunctuation(word_list)     
                word_list_stop=removeStopWords(word_list_punct)
                words_count = collections.Counter(word_list_stop)
                if int(row[0])==1:
                    pos_review_count+=1
                    for word in words_count:   
                        if word in tot_dic:
                            f_value = tot_dic[word]
                            f_value[0] += words_count[word]
                            f_value[1] += 1
                            tot_dic[word] = f_value
                        else:
                            tot_dic[word] = [words_count[word], 1]
                        if word in pos_dic:
                            f_value = pos_dic[word]
                            f_value[0] += words_count[word]
                            f_value[1] += 1
                            pos_dic[word] = f_value
                        else:
                            pos_dic[word]= [words_count[word], 1]

                else: 
                    neg_review_count+=1
                    for word in words_count:
                        word=word
                        if word in tot_dic:
                            f_value = tot_dic[word]
                            f_value[0] += words_count[word]
                            f_value[1] += 1
                            tot_dic[word] = f_value
                        else:
                            tot_dic[word] = [words_count[word], 1]    
                        if word in neg_dic:
                            f_value = neg_dic[word]
                            f_value[0] += words_count[word]
                            f_value[1] += 1                    
                            neg_dic[word] = f_value
                        else:
                            neg_dic[word]= [words_count[word], 1]
    training_file.close()
    
    return pos_dic,neg_dic,tot_dic,total,pos_review_count,neg_review_count

#pos_dic,neg_dic,tot_dic,total,pos_review_count,neg_review_count = classify_train('train.csv')                        

**Positive and negative feature dictionaries:**

The positive and negative dictionary represents a dictionary of key value pairs where the dictionary key is a positive/negative word and the corresponding value is a list where the first element of the list indicates the total occurrence of that particular word in the entire training set and the second element of the list indicates the total number of documents in which the particular word occurs atleast once.

In [44]:
'''Inorder to execute this below code run the above module by removing the commented line in the last and then run this.'''
firstfewpairs_pos = {k: pos_dic[k] for k in sorted(pos_dic.keys())[:20]}
print firstfewpairs_pos
firstfewpairs_neg = {k: neg_dic[k] for k in sorted(neg_dic.keys())[:20]}
print firstfewpairs_neg

{'  for': [1, 1], ' ': [465, 315], '  ': [652, 450], '   ': [9, 7], ' 21': [2, 2], '  and': [2, 2], '  transcendentally': [1, 1], '  you': [1, 1], ' 11  ': [1, 1], '  is': [1, 1], ' 2001 ': [5, 5], ' 60': [1, 1], '  especially': [1, 1], ' 3rd': [1, 1], '  to': [1, 1], ' 13': [1, 1], " 2001 ''": [1, 1], ' 300 ': [2, 2], '  smart': [1, 1], ' &#8211 ': [4, 2]}
{' (uninteresting)  ': [1, 1], ' ': [407, 283], '  ': [637, 436], '   ': [10, 10], ' 21': [1, 1], ' 24 ': [1, 1], '  and': [1, 1], ' 2001  ': [1, 1], '  the': [1, 1], ' (unwatched) ': [1, 1], " 'home": [1, 1], ' &#150 ': [4, 4], ' 28': [1, 1], '  only': [1, 1], ' &#8211 ': [1, 1], ' 1941  ': [1, 1], ' (500)': [1, 1], ' 24  ': [1, 1], ' 27': [1, 1], '  ouch ': [1, 1]}


**Feature selection method - Chi-square**

The x2 test is used in statistics, among other things, to test the independence of two events. More specifically in feature selection we use it to test whether the occurrence of a specific term and the occurrence of a specific class are independent. Thus we estimate the following quantity for each term and we rank them by their score: Below is the formal representation of the chi-square feature selection methodology. We decided to implement the same ideology to our text classification problem and experimented by choosing certain parameter features and hence tested on its prediction accuracy. 
<img src="chi-square formula.png"width=700>
We are also aware of the fact that high scores on x2 indicate that the null hypothesis (H0) of independence should be rejected and thus that the occurrence of the term and class are dependent. If they are dependent then we select the feature for the text classification.

Although from a statistical point of view, using the chi-square feature selection might be problematic as whenever we tend to use this statistical test multiple times, then its said that the probability of getting at least one error increases. However, in text classification it rarely matters whether a few additional terms are added to the feature set or removed from it. Rather, the relative importance of features is important. As long as x2 feature selection only ranks features with respect to their usefulness and is not used to make statements about statistical dependence or independence of variables, we need not be overly concerned that it does not adhere strictly to statistical theory.

Below are the stats to be estimated inorder to apply them in the formula,

-n11 - The total number of reviews where the word occurs and provided the review was labelled positive.

-n10 - The total number of reviews where the word occurs and the review was labelled negative.

-n01 - The total number of reviews where the word does not appear and the review was labelled positive.

-n00 - The total number of reviews where the word does not appear and the review was labelled to be negative. 

In [123]:
'''Feature Selection, Computation of Chi-Squared test statistic, we finally sort the dictionary to obtain the top elements. 
 We will use this module under evaluation function.'''

'''Do not execute this block of code now. This chunk is written here only to show how we calculate the above formula'''

chi_sq={}
for feature in tot_dic:
    if feature in pos_dic:        
        n11=pos_dic[feature][1]
    else:
        n11=0
    if feature in neg_dic:
        n10=neg_dic[feature][1]
    else:
        n10=0
    n01=p_count-n11
    n00=n_count-n10

    chi_sq[feature]=((float) (((n11+n10+n01+n00)*pow((n11*n00-n10*n01),2))/(float)((n11+n01)*(n11+n10)*(n10+n00*n01+n00))))

#Sort Vocab dictionary in descending order of Chi-Squared Statistic
sorted_chi_sq=sorted(chi_sq.iteritems(),key=operator.itemgetter(1),reverse=True)
pos_prior=(float)(p_count)/(total)
neg_prior=(float)(n_count)/(total)

total_vocab=len(tot_dic)

**Feature tuning**

In the below function, we check both the positive and negative feature dictionaries and thereby remove unimportant features present in both dictionaries. Based on the value of k, we pick out top k values from the sorted chi-squared dictionary and name them to be important features to be selected, after which we compare these important features with all the keys present in both the positive and negative dictionaries and hence filter them out. Finally we output the dictionaries using pickle for later usage. 

In [17]:
#Set the value of k to tune the number of features selected
def tune_features(k,sorted_chi_sq,pos_dic,neg_dic,tot_dic):
    #k=6000

    # Delete the unimportant features from the positive and negative dictionaries
    imp_feature_list=[]

    for [key,val] in sorted_chi_sq:
        imp_feature_list.append(key)
        if(len(imp_feature_list)==k):
            break
    for key in pos_dic.keys():
        if key not in imp_feature_list:
            pos_dic.pop(key)
            if key in tot_dic:
                tot_dic.pop(key)
    for key in neg_dic.keys():
        if key not in imp_feature_list:
            neg_dic.pop(key)
            if key in tot_dic:
                tot_dic.pop(key)
    f=open('pos_dictionary.txt','w')
    pickle.dump(pos_dic,f)
    f.close()

    f=open('neg_dictionary.txt','w')
    pickle.dump(neg_dic,f)
    f.close()

    f=open('vocabulary.txt','w')
    pickle.dump(tot_dic,f)
    f.close()
    print 'Lengths of pos,neg dictionaries for k=',k,'after feature extraction',len(pos_dic),len(neg_dic)
    return pos_dic,neg_dic,tot_dic


**Naive Bayes Classification on testing data using Chi-square feature selection strategy**

The Naive Bayesian classifier is based on Bayes’ theorem with independence assumptions between predictors. A Naive Bayesian model is easy to build, with no complicated iterative parameter estimation which makes it particularly useful for very large datasets. Despite its simplicity, the Naive Bayesian classifier often does surprisingly well and is widely used because it often outperforms more sophisticated classification methods. 

Bayes theorem provides a way of calculating the posterior probability, P(c|x), from P(c), P(x), and P(x|c). Naive Bayes classifier assume that the effect of the value of a predictor (x) on a given class (c) is independent of the values of other predictors. This assumption is called class conditional independence.

A formal representation of the naive bayes algorithm is below,

<img src="nb.png"width=800px>


    
    

In [18]:
'''Testing our classification model using naive bayes approach'''

def classify_test(filename,pos_prior,neg_prior):
    #Variable Declarations
    flag=False
    testing_example_count=0
    token_list=[]
    output_list=[]
    total_test_pos_reviews=0
    total_test_neg_reviews=0
    totalfreqpos=0
    totalfreqneg = 0

    #Import the trained model files
    f=open('pos_dictionary.txt','r')
    pos_dic=pickle.load(f)
    f=open('neg_dictionary.txt','r')
    neg_dic=pickle.load(f)

    f=open('vocabulary.txt','r')
    tot_dic=pickle.load(f)

    #Classification 
    with open (filename,'rb') as testing_file:
        testing_data=csv.reader(testing_file)
        for row in testing_data:
            token_pos_prob_list=[]
            token_neg_prob_list=[]
            outputList=[]
            if flag==False:
                flag=True
                i=0
            else:
                testing_example_count+=1
                word_list_split=re.split('\s+',row[1].lower())
                word_list_minus_punct=removePunctuation(word_list_split)
                word_list_minus_stop=removeStopWords(word_list_minus_punct)
                for word in word_list_minus_stop:
                    token_list.append(word)
                    if word in pos_dic:
                        pos_prob_token=(float)(pos_dic[word][0]+1)/(len(tot_dic)+totalfreqpos)
                        token_pos_prob_list.append(math.log(pos_prob_token))
                    else:
                        token_pos_prob_list.append(math.log((float)(1)/(len(tot_dic)+totalfreqpos)))
                    if word in neg_dic:
                        neg_prob_token=(float)(neg_dic[word][0]+1)/(len(tot_dic)+totalfreqneg)
                        token_neg_prob_list.append(math.log(neg_prob_token))
                    else:
                        token_neg_prob_list.append(math.log((float)(1)/(len(tot_dic)+totalfreqneg)))

                pos_prob_total=0
                pos_prob_total = reduce(lambda a,d: a + d,token_pos_prob_list)
                pos_prob_total = pos_prob_total + math.log(pos_prior)
                neg_prob_total = math.log(neg_prior) + (reduce(lambda a, d:a + d,token_neg_prob_list))
                if pos_prob_total>neg_prob_total:
                    review=1
                    total_test_pos_reviews+=1
                else:
                    review=0
                    total_test_neg_reviews+=1
                i+=1
                output_list.append([i, review])
        c = csv.writer(open("result.csv", "wb"))
        c.writerow(['Review-ID', 'Predicted score'])
        for row in output_list:
            c.writerow(row)

        testing_file.close()
        return total_test_pos_reviews,total_test_neg_reviews,testing_example_count

In [19]:
'''Creating a new csv file containing the true label score and reviews of testing dataframe'''
frames2 = [pos_test,neg_test]
result2 = pd.concat(frames2)
result2= result2.sort_index(axis=1)
result2.to_csv("test-accuracy.csv",sep=',')

In [20]:
'''Remove unwanted columns and write data to a new csv file which will be used in the evaluation module to compute accuracy'''
with open("test-accuracy.csv","rb") as source:
    rdr= csv.reader( source )
    with open("test-data.csv","wb") as result:
        wtr= csv.writer( result )
        for r in rdr:
            wtr.writerow( (r[1], r[2]) )

In [21]:
'''Get the true labels for all reviews in the testing data csv file and store them into a python list. This list will be used 
 to compare and hence calculate classification accuracy'''
test_score=[]
with open("test-data.csv","rb") as result:
    rdr= csv.reader( result ) 
    for r in rdr:
        test_score.append(r[0])
test_score.remove('score')

Run this evaluation function by passing a list of parameter values that represent the total number of chi-squared 
features to be considered while trying to classify testing data. Examine the accuracy variation for each parameter change
and hence choose the best tuning parameter. 

In [24]:
'''Run this evaluation function by passing a list of parameter values that represent the total number of chi-squared 
features to be considered while trying to classify testing data. Examine the accuracy variation for each parameter change
and hence choose the best tuning parameter. '''

def evaluation(param):
    for k in param:
        pos_dic,neg_dic,tot_dic,total,p_count,n_count = classify_train("train.csv")
        chi_sq={}
        for feature in tot_dic:
            if feature in pos_dic:        
                n11=pos_dic[feature][1]
            else:
                n11=0
            if feature in neg_dic:
                n10=neg_dic[feature][1]
            else:
                n10=0
            n01=p_count-n11
            n00=n_count-n10

            chi_sq[feature]=((float) (((n11+n10+n01+n00)*pow((n11*n00-n10*n01),2))/(float)((n11+n01)*(n11+n10)*(n10+n00*n01+n00))))

        #Sort Vocab dictionary in descending order of Chi-Squared Statistic
        sorted_chi_sq=sorted(chi_sq.iteritems(),key=operator.itemgetter(1),reverse=True)
        pos_prior=(float)(p_count)/(total)
        neg_prior=(float)(n_count)/(total)

        total_vocab=len(tot_dic)
    
        pos_dic2,neg_dic2,tot_dic2 = tune_features(k,sorted_chi_sq,pos_dic,neg_dic,tot_dic)
        print'Some statistics for a given k=',k
        print 'The vocabulary contains',total_vocab,'words'
        print 'Positive Features Dictionary',len(pos_dic2)
        print 'Negative Features Dictionary',len(neg_dic2)
        print 'Chi-Squared Values',len(tot_dic2)
        print'PRIORS',pos_prior,neg_prior
        print '----------------------------------------------------------'
        print 'Now lets see how our classification model performs'
        total_p_reviews2,total_n_reviews2,t_count2 = classify_test('test-ex2.csv',pos_prior,neg_prior)
        print 'Total Positives: ',total_p_reviews2,'Total Negatives: ',total_n_reviews2
        predicted_score=[]
        with open("result.csv","rb") as result:
            rdr= csv.reader( result ) 
            for r in rdr:
                predicted_score.append(r[1])
        predicted_score.remove('Predicted score')
        final = [i for i, j in zip(test_score, predicted_score) if i == j]
        print 'We get %.4g classification accuracy' % ((len(final)/len(test_score)) * 100)
        print'End -------------------------of--------------------------------statistics'
        

In [25]:
'''Call evaluation function by passing a list of parameter values. '''

evaluation([5000,6000,7000,8000,9000])

C:\Users\Praveen\Anaconda\lib\site-packages\IPython\kernel\__main__.py:18: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Praveen\Anaconda\lib\site-packages\IPython\kernel\__main__.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Lengths of pos,neg dictionaries for k= 5000 after feature extraction 3737 3885
Some statistics for a given k= 5000
The vocabulary contains 47991 words
Positive Features Dictionary 3737
Negative Features Dictionary 3885
Chi-Squared Values 5000
PRIORS 0.5 0.5
----------------------------------------------------------
Now lets see how our classification model performs
Total Positives:  11905 Total Negatives:  11965
We get 83.78 classification accuracy
End -------------------------of--------------------------------statistics
Lengths of pos,neg dictionaries for k= 6000 after feature extraction 4431 4574
Some statistics for a given k= 6000
The vocabulary contains 47991 words
Positive Features Dictionary 4431
Negative Features Dictionary 4574
Chi-Squared Values 6000
PRIORS 0.5 0.5
----------------------------------------------------------
Now lets see how our classification model performs
Total Positives:  11889 Total Negatives:  11981
We get 84.55 classification accuracy
End ----------------

**Inference**

By looking at the performance of our classifier, we could say that the top accuracy has been obtained for k=9000. Even otherwise the accuracy seems to increase on a linear fashion as we increase the total number of features to include. This approach of using chi-squared feature selection method is also proved to have positive impact during classification as the results are much better when compared to our classification performance carried out by Logistic regression and the multinomial naive bayes models. However, In our case since we have about 47991 words present in the vocabulary set, we could select the number of influential features to be anywhere between 8000 and 10000 to thereby achieve optimal classification. Increasing the tuning parameter even more might result in biased performance. Overall we would like to say that, given the negativity involved in using the chi-square test statistic measure as a feature selection strategy, the naive bayes classifier performs well to meet our expectation. But still while choosing the number of features 'k', we would recommend choosing 'k' by considering the length of the vocabulary and the size of the training set. Choosing more number of 'k' features just to attain better accuracy may lead to extreme bias and memory constraints.  